In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split


In [2]:
def loadData(filePath):
    data = pd.read_csv(filePath)
    # 去除Cabin、Ticket、Name三列，这三列不做分析
    data.drop(columns = ['Cabin', 'Ticket', 'Name'], axis = 1, inplace = True)
    # 处理缺失值：Age使用平均值填充，Embarked使用出现最多的填充
    data['Age'].fillna(value=data['Age'].median(), inplace = True)
    maxEmbarked = data['Embarked'].value_counts().sort_values(ascending = False).index[0]
    data['Embarked'].fillna(value = maxEmbarked, inplace = True)
    # 修改male为1，female为0
    data.loc[data['Sex']=='male', 'Sex'] = 1
    data.loc[data['Sex']=='female', 'Sex'] = 0
    # 修改C为0，Q为1，S为2
    data.loc[data['Embarked']=='C', 'Embarked'] = 0
    data.loc[data['Embarked']=='Q', 'Embarked'] = 1
    data.loc[data['Embarked']=='S', 'Embarked'] = 2
    # data[['Sex','Embarked']] = data[['Sex','Embarked']].apply(pd.to_numeric)
    data[['Sex','Embarked']] = data[['Sex','Embarked']].astype(np.int64)
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
    tag = ['Survived']
    # 转为矩阵
    if 'Survived' in data.columns:
        X = np.array(data[features])
        y = np.array(data[tag]).flatten()
        y[y==0] = -1
        return X, y
    else:
        X = np.array(data[features])
        return X

In [3]:
# 标准化
def norm(X_data):
    mu = X_data.mean(axis = 0)
    sigma = X_data.std(axis = 0)
    sigma[sigma==0] = 1
    X_norm = (X_data - mu) / sigma
    return X_norm

In [67]:
class SVM:
    def __init__(self, X, y, kernel = 'rbf', C = 1, tol = 1e-3, sigma = 5, degree = 3, max_iter = 100):
        '''
            kernel: 核函数
            C: 惩罚系数
            tol: svm结束标准的精度
        '''
        self.kernel = kernel
        self.C = C
        self.tol = tol
        self.max_iter = max_iter
        self.sigma = sigma     # 高斯核std
        self.degree = degree   # 多项式度数
        self.X = X
        self.y = y
        self.N = X.shape[0]    # 训练样本量
        self.m = X.shape[1]    # 特征数
        self.K = None
        self.calK()
        self.b = 0
        self.alpha = np.zeros(self.N)
#         self.alpha = np.array([0 for _ in range(self.N)]) # 这样写alpha就变成int了
        self.E = np.array([-y[i] for i in range(self.N)], dtype = np.float64)  # E_i = (sum_{j=1}^N \alpha_j y_j Kij + b) - y_i
        self.supportVector = []  # 支持向量点
        self.fit()
    def RBFKernel(self, x1, x2):
        return np.exp(-np.linalg.norm(x1-x2)**2/(2 * self.sigma ** 2))
    def LinearKernel(self, x1,x2):
        return x1@x2
    def PolyKernel(self, x1, x2):
        return np.power((x1@x2+1), self.degree)
    def calK(self):
        # 计算核矩阵Kij
        if self.kernel == 'rbf':
            self.kernelFunc = self.RBFKernel
        elif self.Kernel == 'linear':
            self.kernelFunc = self.LinearKernel
        elif self.Kernel == 'poly':
            self.kernelFunc = self.PolyKernel
        else:
            self.kernelFunc = RBFKernel
        self.K = np.zeros(shape = (self.N,self.N))
        # 只计算上三角矩阵 Kij = kji
        for i in range(self.N):
            for j in range(i, self.N):
                self.K[i,j] = self.kernelFunc(self.X[i], self.X[j])
                self.K[j,i] = self.K[i,j]
        print(self.K)
    # 式7.104
    def g(self, i):
        # 以下两种写法会导致最后的准确率不同？？？
#         return self.b + np.sum([self.alpha[j] * self.y[j] * self.K[i,j] for j in range(self.N)])
        return self.b + np.sum([self.alpha[j] * self.y[j] * self.K[i,j] for j in (np.arange(self.alpha.shape[0])[self.alpha!=0]) ] )
    def satisfyKKT(self, i):
        # 在self.tol范围内判断是否满足KKT条件
        gxi = self.g(i)
        if np.abs(self.alpha[i]) <= self.tol and self.y[i] *  gxi >= 1 - self.tol:
            return True
        elif self.alpha[i] < self.C + self.tol and self.alpha[i] > 0 - self.tol and np.abs(self.y[i] * gxi - 1) <= self.tol:
            return True
        elif np.abs(self.alpha[i] - self.C) <= self.tol and self.y[i] * gxi <= 1 + self.tol:
            return True
        return False
    def get_alpha2(self, i):
#         return np.argmax(np.abs(np.array([self.E[i] - self.E[j] for j in range(self.N)])))
        return np.argmax(np.abs(self.E[i] - self.E))
    def cal_E(self, i):
        return self.g(i) - self.y[i]
    def simple_SMO(self, max_epoch = 100):
        # 在self.tol精度下求解
        for epoch in range(max_epoch):
            upd_num = 0
            # 简化版本，直接枚举第一个变量
            for i in range(self.N):
                # 满足KKT条件则找下一个
                if self.satisfyKKT(i):
                    continue
                # 寻找第二个变量
                j = self.get_alpha2(i)
                K11, K22, K12, K21 = self.K[i,i], self.K[j,j], self.K[i,j], self.K[j,i]
                eta = K11 + K22 - 2 * K12 # 式7.107
                
                ##########################################################
                if eta == 0:
                    continue
                alpha1_old, y1, E1 = self.alpha[i], self.y[i], self.E[i]
                alpha2_old, y2, E2 = self.alpha[j], self.y[j], self.E[j]
                b_old = self.b
                # 式7.106计算alpha2_new_unc
                alpha2_new_unc = alpha2_old + y2 * (E1 - E2) / eta
                # 计算参数范围
                if y1 != y2:
                    L, H = max(0, alpha2_old - alpha1_old), min(self.C, self.C + alpha2_old - alpha1_old)
                else:
                    L, H = max(0, alpha1_old + alpha2_old - self.C), min(self.C, alpha1_old + alpha2_old)
                # 剪辑最优解 式7.108
                if alpha2_new_unc > H:
                    alpha2_new = H
                elif alpha2_new_unc < L:
                    alpha2_new = L
                else:
                    alpha2_new = alpha2_new_unc
                # 式7.109 更新alpha1_new
                alpha1_new = alpha1_old + y1 * y2 * (alpha2_old - alpha2_new)
                # 判断改变量是否超过精度范围
                if np.abs(alpha2_new - alpha2_old) > self.tol:
                    upd_num += 1
                # 式7.115和7.116更新b1_new和b2_new
                b1_new = -E1 - y1 * K11 * (alpha1_new - alpha1_old) - y2 * K21 * (alpha2_new - alpha2_old) + b_old
                b2_new = -E2 - y1 * K12 * (alpha1_new - alpha1_old) - y2 * K22 * (alpha2_new - alpha2_old) + b_old
                # 更新b_new
                if 0 < alpha1_new and alpha1_new < self.C:
                    b_new = b1_new
                elif 0 < alpha2_new and alpha2_new < self.C:
                    b_new = b2_new
                else:
                    b_new = (b1_new + b2_new) / 2
                self.alpha[i], self.alpha[j] = alpha1_new, alpha2_new
                self.b = b_new
                self.E[i], self.E[j] = self.cal_E(i), self.cal_E(j)
            print('SMO epoch%d, max epoch = %d, updated number = %d.' % (epoch + 1, max_epoch, upd_num))
            if upd_num == 0:
                break
    
    def sgn(self, x):
        return 1 if x >= 0 else -1
    def fit(self):
#         SMO算法解对偶问题得到alpha*
        self.simple_SMO(max_epoch = self.max_iter)
        for i in range(self.N):
            if self.alpha[i] > 0:
                self.supportVector.append(i)
        self.supportVector = np.array(self.supportVector, dtype = np.int32)
#         
    def predict(self, X):
        pred_y = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            pred_y[i] = self.sgn(np.sum([self.alpha[j] * self.y[j] * self.kernelFunc(X[i], self.X[j]) for j in self.supportVector]) + self.b)
        pred_y[pred_y == 0] = -1
        return pred_y.astype(np.int32)

In [5]:
trainFilePath = '../titanic/train.csv'
X, y = loadData(trainFilePath)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
X_train, X_valid = norm(X_train), norm(X_valid)

In [92]:

start = time.time()
model = SVM(X_train, y_train, kernel = 'rbf', C = 1, tol = 1e-5, sigma = 2, max_iter = 100)
end = time.time()
print('training time : %fs' % (end - start))

[[1.         0.70248169 0.11067247 ... 0.07562975 0.36636061 0.46337644]
 [0.70248169 1.         0.05808486 ... 0.06083561 0.59069416 0.80383858]
 [0.11067247 0.05808486 1.         ... 0.00563144 0.12412196 0.12673476]
 ...
 [0.07562975 0.06083561 0.00563144 ... 1.         0.05064295 0.03765388]
 [0.36636061 0.59069416 0.12412196 ... 0.05064295 1.         0.7605525 ]
 [0.46337644 0.80383858 0.12673476 ... 0.03765388 0.7605525  1.        ]]
SMO epoch1, max epoch = 100, updated number = 171.
SMO epoch2, max epoch = 100, updated number = 78.
SMO epoch3, max epoch = 100, updated number = 95.
SMO epoch4, max epoch = 100, updated number = 59.
SMO epoch5, max epoch = 100, updated number = 66.
SMO epoch6, max epoch = 100, updated number = 66.
SMO epoch7, max epoch = 100, updated number = 62.
SMO epoch8, max epoch = 100, updated number = 79.
SMO epoch9, max epoch = 100, updated number = 69.
SMO epoch10, max epoch = 100, updated number = 98.
SMO epoch11, max epoch = 100, updated number = 77.
SMO

In [93]:
y_pred = model.predict(X_valid)
acc = (y_pred == y_valid).sum() / y_pred.shape[0]
print('acc = %f.' % (acc))

# model = SVM(X_train, y_train, kernel = 'rbf', C = 1, tol = 1e-3, sigma = 2, max_iter = 20) acc = 0.798883.
# model = SVM(X_train, y_train, kernel = 'rbf', C = 1, tol = 1e-5, sigma = 2, max_iter = 100) acc = 0.821229.
# model = SVM(X_train, y_train, kernel = 'rbf', C = 1, tol = 1e-5, sigma = 3, max_iter = 100) acc = 0.804469.
# model = SVM(X_train, y_train, kernel = 'rbf', C = 1, tol = 1e-4, sigma = 2, max_iter = 200) acc = 0.815642.

acc = 0.821229.


In [100]:
testFilePath = '../titanic/test.csv'
X_test = loadData(testFilePath)
X_test = norm(X_test)
y_pred = model.predict(X_test)
y_pred[y_pred==-1] = 0
savePath = './result.csv'
PassengerId = np.array(pd.read_csv(testFilePath)['PassengerId'])
ans = pd.DataFrame({'PassengerId': PassengerId, 'Survived' : y_pred})
ans.to_csv(savePath, index = False)

In [15]:
# from sklearn.svm import SVC
# model = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#     max_iter=-1, probability=False, random_state=None, shrinking=True,
#     tol=0.001, verbose=False)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_valid)
# acc = (y_pred == y_valid).sum() / y_pred.shape[0]
# print(acc)

0.8100558659217877
